## Importações

In [137]:
import os
from itertools import product

import numpy as np
import pandas as pd
from scipy.stats import zscore

from sklearn.datasets import make_regression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [138]:
%pip install XlsxWriter

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Plano de Experimentação

### 1. Carregar o dataset

In [139]:
df = pd.read_csv('../../../data/pre-processado/por-municipio/completo/dados-manaus-preprocessado.csv') 

# Retirando antes dos anos 2000 pois dados de clima ausentes, e inserir pela média prejudicou o modelo
df = df[df['ano']>=2000]

# Retirando pois variáveis não necessárias para o modelo, visto que são iguais para todas as linhas
df = df.drop(columns=['municipio','uf'])

### 2. Divisão do Dataset

In [140]:
X = df.drop(columns='vazao')
y = df['vazao'] # Separa o target
cat_cols = X.select_dtypes(include='object').columns.tolist()

### 3. Pré-processamentos

In [141]:
def remover_outliers(df, colunas, threshold=3):
    z_scores = np.abs(zscore(df[colunas]))
    mask = (z_scores < threshold).all(axis=1)
    return df[mask]

### 4. Parâmetros de variação

In [142]:
classificacoes = ['2 classificações', '3 classificações']
validacao = ['k=3', 'k=10']
criterios = ['squared_error']
n_arvores = [50, 100]
etas = [0.1, 0.3]
depths = [3, 5]
normalizadores = [StandardScaler(), MinMaxScaler(), RobustScaler()]
usar_outliers = [True, False]


### 5. Cenários

In [143]:
cenarios_rfr = list(product(classificacoes, criterios, n_arvores, validacao, normalizadores, usar_outliers))
cenarios_xgb = list(product(classificacoes, etas, depths, validacao, normalizadores, usar_outliers))

### 4. Random Forest Regressor

In [ ]:
resultados_rfr = []

for i, (classe, criterio, n, val, norm, outlier_flag) in enumerate(cenarios_rfr):
    X_temp = X.copy()
    y_temp = y.copy()
    num_cols = X_temp.select_dtypes(include=['int64', 'float64']).columns.tolist()

    if outlier_flag:
        df_completo = X_temp.copy()
        df_completo["target"] = y_temp
        df_sem_outliers = remover_outliers(df_completo, num_cols)
        X_temp = df_sem_outliers.drop(columns="target")
        y_temp = df_sem_outliers["target"]

    # Se o dataset ficou vazio após remoção de outliers, pula o cenário
    if X_temp.shape[0] < 2:
        continue

    preprocessador = ColumnTransformer([
        ("scale", norm, num_cols)
    ], remainder='passthrough')

    X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=i+100)

    modelo = Pipeline([
        ("prep", preprocessador),
        ("clf", RandomForestRegressor(n_estimators=n, criterion=criterio, random_state=42))
    ])
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    resultados_rfr.append([
        i + 1,
        'X' if classe == '2 classificações' else '',
        'X' if classe == '3 classificações' else '',
        criterio,
        n,
        'X' if val == 'k=3' else '',
        'X' if val == 'k=10' else '',
        'X' if outlier_flag else '',
        type(norm).__name__.replace('Scaler', ''),
        mse, mae, r2
    ])

### 5. XGBoost Regressor

In [ ]:
resultados_xgbr = []

for i, (classe, eta, depth, val, norm, outlier_flag) in enumerate(cenarios_xgb):
    X_temp = X.copy()
    y_temp = y.copy()
    num_cols = X_temp.select_dtypes(include=['int64', 'float64']).columns.tolist()

    if outlier_flag:
        df_completo = X_temp.copy()
        df_completo["target"] = y_temp
        df_sem_outliers = remover_outliers(df_completo, num_cols)
        X_temp = df_sem_outliers.drop(columns="target")
        y_temp = df_sem_outliers["target"]

    if X_temp.shape[0] < 2:
        continue

    preprocessador = ColumnTransformer([
        ("scale", norm, num_cols)
    ], remainder='passthrough')

    X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=i+200)

    modelo = Pipeline([
        ("prep", preprocessador),
        ("clf", XGBRegressor(learning_rate=eta, max_depth=depth, random_state=42))
    ])
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    resultados_xgbr.append([
        i + 1,
        'X' if classe == '2 classificações' else '',
        'X' if classe == '3 classificações' else '',
        eta,
        depth,
        'X' if val == 'k=3' else '',
        'X' if val == 'k=10' else '',
        'X' if outlier_flag else '',
        type(norm).__name__.replace('Scaler', ''),
        mse, mae, r2
    ])

### 7. Dataframe com Resultados

In [ ]:
df_rfr = pd.DataFrame(resultados_rfr, columns=[
    'Cenário', '2 classificações', '3 classificações', 'Critério',
    'Nº Árvores', 'Validação k=3', 'Validação k=10',
    'Remoção de Outliers', 'Normalização',
    'MSE', 'MAE', 'R²'
])

df_xgbr = pd.DataFrame(resultados_xgbr, columns=[
    'Cenário', '2 classificações', '3 classificações', 'Taxa de Aprendizado',
    'Profundidade', 'Validação k=3', 'Validação k=10',
    'Remoção de Outliers', 'Normalização',
    'MSE', 'MAE', 'R²'
])

### 7. Formatação do Dataframe

In [ ]:
# XGBoost
df_xgbr['Taxa 0.1'] = df_xgbr['Taxa de Aprendizado'].apply(lambda x: 'X' if x == 0.1 else '')
df_xgbr['Taxa 0.3'] = df_xgbr['Taxa de Aprendizado'].apply(lambda x: 'X' if x == 0.3 else '')
df_xgbr['Profundidade 3'] = df_xgbr['Profundidade'].apply(lambda x: 'X' if x == 3 else '')
df_xgbr['Profundidade 5'] = df_xgbr['Profundidade'].apply(lambda x: 'X' if x == 5 else '')
df_xgbr['Standard'] = df_xgbr['Normalização'].apply(lambda x: 'X' if x == 'Standard' else '')
df_xgbr['MinMax'] = df_xgbr['Normalização'].apply(lambda x: 'X' if x == 'MinMax' else '')
df_xgbr['Robust'] = df_xgbr['Normalização'].apply(lambda x: 'X' if x == 'Robust' else '')

# Random Forest
df_rfr['Standard'] = df_rfr['Normalização'].apply(lambda x: 'X' if x == 'Standard' else '')
df_rfr['MinMax'] = df_rfr['Normalização'].apply(lambda x: 'X' if x == 'MinMax' else '')
df_rfr['Robust'] = df_rfr['Normalização'].apply(lambda x: 'X' if x == 'Robust' else '')

### 8. Dataframes finais

In [ ]:
# Random Forest
df_rfr = df_rfr[[
    'Cenário', '2 classificações', '3 classificações', 
    'Standard', 'MinMax', 'Robust',
    'Critério',
    'Nº Árvores', 'Validação k=3', 'Validação k=10',
    'Remoção de Outliers',
    'MSE', 'MAE', 'R²'
]]

# XGBoost
df_xgbr = df_xgbr[[
    'Cenário', '2 classificações', '3 classificações', 
    'Remoção de Outliers', 
    'Standard', 'MinMax', 'Robust'
    'Taxa 0.1', 'Taxa 0.3', 
    'Profundidade 3', 'Profundidade 5',
    'Validação k=3', 'Validação k=10',
    'MSE', 'MAE', 'R²'
]]

TypeError: list indices must be integers or slices, not list

### 6. Exportação

In [ ]:
with pd.ExcelWriter('../../../data/plano-de-experimentacao/plano_de_experimentacao.xlsx', engine='xlsxwriter') as writer:
    df_rfr.to_excel(writer, sheet_name='Random Forest Regressor', index=False)
    df_xgbr.to_excel(writer, sheet_name='XGBoost Regressor', index=False)

print("Excel gerado com sucesso!")

AttributeError: 'list' object has no attribute 'to_excel'